In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

from tqdm.auto import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
pretrain_features = {
    "learning_rate": 0.1,
    "batch_size": 256,
    "epochs": 1000,
    "eval_size": 4*256,
    "momentum": 0.005,
    "weight_decay": 0.0001,
}

In [4]:
def load_data():
    """
    This function loads the data from the csv files and returns it as numpy arrays.

    input: None
    
    output: x_pretrain: np.ndarray, the features of the pretraining set
            y_pretrain: np.ndarray, the labels of the pretraining set
            x_train: np.ndarray, the features of the training set
            y_train: np.ndarray, the labels of the training set
            x_test: np.ndarray, the features of the test set
    """
    x_pretrain = pd.read_csv("./pretrain_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_pretrain = pd.read_csv("./pretrain_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_train = pd.read_csv("./train_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1).to_numpy()
    y_train = pd.read_csv("./train_labels.csv.zip", index_col="Id", compression='zip').to_numpy().squeeze(-1)
    x_test = pd.read_csv("./test_features.csv.zip", index_col="Id", compression='zip').drop("smiles", axis=1)
    scaler = StandardScaler()
    x_pretrain = scaler.fit_transform(x_pretrain)
    x_train = scaler.transform(x_train)
    x_test_transed = scaler.transform(x_test)
    x_test[x_test.columns] = x_test_transed

    return x_pretrain, y_pretrain, x_train, y_train, x_test

In [5]:
x_pretrain, y_pretrain, x_train, y_train, x_test = load_data()

c:\Users\linan\.conda\envs\DL\lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [6]:
class AE(nn.Module):
    """
    The model class, which defines our feature extractor used in pretraining.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(1000, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256))

        self.decoder = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512, 1000),
            nn.Sigmoid()
            )
            
        for m in self.modules():
            if isinstance(m, nn.Linear):    
                nn.init.xavier_uniform_(m.weight)
        
        
    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    
    def encode(self, x):
        return self.encoder(x)

In [7]:
eval_size = pretrain_features["eval_size"]
batch_size = pretrain_features["batch_size"]
ae_model = AE()
ae_model.train()
ae_model.to(device)

def train_autoencoder():
    x_tr, x_val, y_tr, y_val = train_test_split(x_pretrain, y_pretrain, test_size=eval_size, random_state=0, shuffle=True)
    x_tr, x_val = torch.tensor(x_tr, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)
    y_tr, y_val = torch.tensor(y_tr, dtype=torch.float), torch.tensor(y_val, dtype=torch.float)
    train_dataset = torch.utils.data.TensorDataset(x_tr, y_tr)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    # optimizer = torch.optim.Adam(model.parameters(), lr=pretrain_features['learning_rate'], weight_decay=pretrain_features['weight_decay'])
    optimizer = torch.optim.SGD(ae_model.parameters(), lr=pretrain_features['learning_rate'], momentum=pretrain_features['momentum'], weight_decay=pretrain_features['weight_decay'])
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5, verbose=True)
    criterion = nn.MSELoss()
    epochs = pretrain_features['epochs']
    for epoch in tqdm(range(epochs)):
        loss_tr = 0
        loss_val = 0
        for [x, _] in train_loader:
            optimizer.zero_grad()
            x = x.to(device)
            predictions = ae_model(x)
            loss = criterion(predictions, x)
            loss.backward()
            optimizer.step()
            loss_tr += loss.item() * len(x)
        loss_tr /= len(train_loader.dataset)
        for [x, _] in val_loader:
            x = x.to(device)
            predictions = ae_model(x)
            loss = criterion(predictions, x)
            loss_val += loss.item() * len(x)
        loss_val /= len(val_loader.dataset)
        scheduler.step(loss_val)
        print(f"Epoch {epoch+1}: train loss: {loss_tr}, val loss: {loss_val}")
        if(optimizer.param_groups[0]['lr'] < 1e-6):
            print(f"Early stop at epoch {epoch+1}")
            break

train_autoencoder()

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 1: train loss: 1.2468429364943108, val loss: 1.3335063755512238
Epoch 2: train loss: 1.236247278630675, val loss: 1.3234477639198303
Epoch 3: train loss: 1.2262322879702154, val loss: 1.314057171344757
Epoch 4: train loss: 1.2166956142442984, val loss: 1.3049940466880798
Epoch 5: train loss: 1.2075564524037743, val loss: 1.2961877286434174
Epoch 6: train loss: 1.1986793045591038, val loss: 1.2876657545566559
Epoch 7: train loss: 1.1899396043242836, val loss: 1.2791130542755127
Epoch 8: train loss: 1.1811766089352467, val loss: 1.2704075574874878
Epoch 9: train loss: 1.1722350078484312, val loss: 1.2615307569503784
Epoch 10: train loss: 1.1629440669178457, val loss: 1.2519768178462982
Epoch 11: train loss: 1.153098899529129, val loss: 1.2418641149997711
Epoch 12: train loss: 1.1424624092708964, val loss: 1.2306165099143982
Epoch 13: train loss: 1.130789080736334, val loss: 1.2185284793376923
Epoch 14: train loss: 1.1178619136455128, val loss: 1.204876333475113
Epoch 15: train loss

In [8]:
class Net(nn.Module):
    """
    The model class, which defines our feature extractor used in pretraining.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        # TODO: Define the architecture of the model. It should be able to be trained on pretraing data 
        # and then used to extract features from the training and test data.
        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 1)

        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.3)

        nn.init.xavier_normal_(self.fc1.weight)
        nn.init.xavier_normal_(self.fc2.weight)
        nn.init.xavier_normal_(self.fc3.weight)


    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        # TODO: Implement the forward pass of the model, in accordance with the architecture 
        # defined in the constructor.
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x
    
    def encode(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        return x

In [9]:
pretrained_x = ae_model.encoder(torch.tensor(x_pretrain, dtype=torch.float).to(device)).detach().cpu().numpy()

In [10]:
# model declaration
nn_model = Net()
nn_model.to(device)
nn_model.train()
    
def train_nn():
    x_tr, x_val, y_tr, y_val = train_test_split(pretrained_x, y_pretrain, test_size=eval_size, random_state=0, shuffle=True)
    x_tr, x_val = torch.tensor(x_tr, dtype=torch.float), torch.tensor(x_val, dtype=torch.float)
    y_tr, y_val = torch.tensor(y_tr, dtype=torch.float), torch.tensor(y_val, dtype=torch.float)
    train_dataset = torch.utils.data.TensorDataset(x_tr, y_tr)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    # TODO: Implement the training loop. The model should be trained on the pretraining data. Use validation set 
    # to monitor the loss.
    optimizer = torch.optim.Adam(nn_model.parameters(), lr=0.001, weight_decay=0.0001)    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5, verbose=True)
    criterion = nn.MSELoss()
    epochs = 500
    for epoch in tqdm(range(epochs)):
        loss_tr = 0
        loss_val = 0
        for [x, y] in train_loader:
            optimizer.zero_grad()
            x = x.to(device)
            y = y.to(device)
            predictions = nn_model(x).squeeze(-1)
            loss = criterion(predictions, y)
            loss.backward()
            optimizer.step()
            loss_tr += loss.item() * len(x)
        loss_tr /= len(train_loader.dataset)
        for [x, y] in val_loader:
            x = x.to(device)
            y = y.to(device)
            predictions = nn_model(x).squeeze(-1)
            loss = criterion(predictions, y)
            loss_val += loss.item() * len(x)
        loss_val /= len(val_loader.dataset)
        scheduler.step(loss_val)
        print(f"Epoch {epoch+1}: train loss: {loss_tr}, val loss: {loss_val}")
        if(optimizer.param_groups[0]['lr'] < 1e-6):
            print(f"Early stop at epoch {epoch+1}")
            break

train_nn()

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch 1: train loss: 1.1516941652621993, val loss: 0.6298345625400543
Epoch 2: train loss: 0.5363350652063333, val loss: 0.4943477138876915
Epoch 3: train loss: 0.47830825309977737, val loss: 0.46450424939393997
Epoch 4: train loss: 0.4339446583650191, val loss: 0.42630717158317566
Epoch 5: train loss: 0.4121505918942102, val loss: 0.3848237693309784
Epoch 6: train loss: 0.3806552706162784, val loss: 0.39149825274944305
Epoch 7: train loss: 0.34093005140791216, val loss: 0.31563978642225266
Epoch 8: train loss: 0.3052934439635596, val loss: 0.29482533037662506
Epoch 9: train loss: 0.280585488494959, val loss: 0.27971335500478745
Epoch 10: train loss: 0.2593205975127041, val loss: 0.26149865612387657
Epoch 11: train loss: 0.23717140270794423, val loss: 0.20947310328483582
Epoch 12: train loss: 0.2148386099220919, val loss: 0.21185561269521713
Epoch 13: train loss: 0.19455166818249736, val loss: 0.1873474456369877
Epoch 14: train loss: 0.1777015946652133, val loss: 0.1461114976555109
Epo

In [11]:
featured_x_train = ae_model.encoder(torch.tensor(x_train, dtype=torch.float).to(device))
featured_x_train = nn_model.encode(featured_x_train).detach().cpu().numpy()
scaler = StandardScaler()
featured_x_train = scaler.fit_transform(featured_x_train)

In [12]:
class onelayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(16, 1)
        nn.init.xavier_normal_(self.fc1.weight)
    
    def forward(self, x):
        x = self.fc1(x)
        return x

In [13]:
def get_regression_model(X, y):

    class Model(nn.Module):
        """
        The model class, which defines our feature extractor used in pretraining.
        """
        def __init__(self):
            """
            The constructor of the model.
            """
            super().__init__()
            # TODO: Define the architecture of the model. It should be able to be trained on pretraing data 
            # and then used to extract features from the training and test data.
            self.fc3 = nn.Linear(16, 1)

            # nn.init.kaiming_uniform_(self.fc3.weight, nonlinearity='relu')
            nn.init.xavier_normal_(self.fc3.weight)

        def forward(self, x):
            """
            The forward pass of the model.

            input: x: torch.Tensor, the input to the model

            output: x: torch.Tensor, the output of the model
            """
            # TODO: Implement the forward pass of the model, in accordance with the architecture 
            # defined in the constructor.
            x = self.fc3(x)
            return x
    
    """
    This function returns the regression model used in the pipeline.

    input: None

    output: model: sklearn compatible model, the regression model
    """
    # TODO: Implement the regression model. It should be able to be trained on the features extracted
    # by the feature extractor.
    model = Model()
    model.to(device)
    model.train()

    x = torch.tensor(X, dtype=torch.float)
    # x = X.clone().detach()
    x = x.to(device)
    y = torch.tensor(y, dtype=torch.float)
    y = y.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=5, verbose=True)
    criterion = nn.MSELoss()
    epochs = 10000
    for epoch in tqdm(range(epochs)):
        optimizer.zero_grad()
        predictions = model(x).squeeze(-1)
        loss = criterion(predictions, y)
        loss.backward()
        optimizer.step()
        scheduler.step(loss)
        if(epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}: train loss: {loss}")
        if(optimizer.param_groups[0]['lr'] < 1e-7):
            print(f"Early stop at epoch {epoch+1}, loss: {loss}")
            break

    return model

one_model = get_regression_model(featured_x_train, y_train)

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch 10: train loss: 3.026984214782715
Epoch 20: train loss: 2.976902961730957
Epoch 30: train loss: 2.936845064163208
Epoch 40: train loss: 2.895547389984131
Epoch 50: train loss: 2.8560314178466797
Epoch 60: train loss: 2.817919969558716
Epoch 70: train loss: 2.78074312210083
Epoch 80: train loss: 2.744584798812866
Epoch 90: train loss: 2.70926570892334
Epoch 100: train loss: 2.6746826171875
Epoch 110: train loss: 2.640730619430542
Epoch 120: train loss: 2.6073248386383057
Epoch 130: train loss: 2.574399471282959
Epoch 140: train loss: 2.5419046878814697
Epoch 150: train loss: 2.5098044872283936
Epoch 160: train loss: 2.478074312210083
Epoch 170: train loss: 2.4466981887817383
Epoch 180: train loss: 2.415666103363037
Epoch 190: train loss: 2.3849713802337646
Epoch 200: train loss: 2.3546102046966553
Epoch 210: train loss: 2.3245797157287598
Epoch 220: train loss: 2.294877767562866
Epoch 230: train loss: 2.2655022144317627
Epoch 240: train loss: 2.2364513874053955
Epoch 250: train lo

In [14]:
result_file = "results-bbbb.csv"

In [15]:
y_pred = np.zeros(x_test.shape[0])
featured_x_test = nn_model.encode(ae_model.encoder(torch.tensor(x_test.to_numpy(), dtype=torch.float).to(device)))
featured_x_test = scaler.transform(featured_x_test.detach().cpu().numpy())
featured_x_test = torch.tensor(featured_x_test, dtype=torch.float).to(device)
y_pred = one_model(featured_x_test).squeeze(-1).detach().cpu().numpy()

assert y_pred.shape == (x_test.shape[0],)
y_pred = pd.DataFrame({"y": y_pred}, index=x_test.index)
y_pred.to_csv(result_file, index_label="Id")
print(f"Predictions saved to {result_file}, all done!")

Predictions saved to results-bbbb.csv, all done!
